In [2]:
import pandas as pd

In [47]:
simple_qa_samples = {}
model_names = ["gpt-4o-mini", "gpt-4o", "claude-3.5-sonnet", "gemini-v1.5-flash",
               "geminipro", "nvidia-llama31-70", "nvidia-llama31-405"]
for model_name in model_names:        
    simple_qa_samples[model_name+"-agent"] = pd.read_json("eval_samples/simple_qa_250_{}-agent.jsonl".format(model_name), lines=True)
    if model_name == "claude-3.5-sonnet":
        simple_qa_samples[model_name+"-agent"] = simple_qa_samples[model_name+"-agent"].iloc[:120]

simple_qa_samples['gemini-v1.5-flash-agent'] = pd.read_json("eval_samples/simple_qa_250-gemini-v1.5-flash-agent-ablation.jsonl", lines=True)

In [48]:
web_search_invoked = {}
for model_name in simple_qa_samples.keys():
    web_search_invoked[model_name] = float((simple_qa_samples[model_name]["search_done"]).mean())
web_search_invoked

{'gpt-4o-mini-agent': 0.784,
 'gpt-4o-agent': 0.78,
 'claude-3.5-sonnet-agent': 0.8916666666666667,
 'gemini-v1.5-flash-agent': 0.628,
 'geminipro-agent': 0.8113207547169812,
 'nvidia-llama31-70-agent': 0.624,
 'nvidia-llama31-405-agent': 0.264}

In [49]:
simple_qa_results = {}
model_names = ["gpt-4o-mini", "gpt-4o", "claude-3.5-sonnet", "gemini-v1.5-flash",
               "geminipro", "nvidia-llama31-70", "nvidia-llama31-405"]
for model_name in model_names:
    simple_qa_results[model_name] = pd.read_json("eval_results/simple_qa_250_{}_results.jsonl".format(model_name), lines=True)
    if model_name == "claude-3.5-sonnet":
        simple_qa_results[model_name] = simple_qa_results[model_name].iloc[:120]
for model_name in model_names:
    simple_qa_results[model_name+"-agent"] = pd.read_json("eval_results/simple_qa_250_{}-agent_results.jsonl".format(model_name), lines=True)
    if model_name == "claude-3.5-sonnet":
        simple_qa_results[model_name+"-agent"] = simple_qa_results[model_name+"-agent"].iloc[:120]

simple_qa_results["gemini-v1.5-flash-agent"] = pd.read_json("eval_results/simple_qa_250-gemini-v1.5-flash-agent-ablation_results.jsonl", lines=True)

In [50]:
accuracy = {}
for model_name in simple_qa_results.keys():
    model_accuracy = float((simple_qa_results[model_name]["grade"] == "A").mean())
    accuracy[model_name] = model_accuracy

accuracy

{'gpt-4o-mini': 0.1,
 'gpt-4o': 0.44,
 'claude-3.5-sonnet': 0.2916666666666667,
 'gemini-v1.5-flash': 0.112,
 'geminipro': 0.2459016393442623,
 'nvidia-llama31-70': 0.152,
 'nvidia-llama31-405': 0.188,
 'gpt-4o-mini-agent': 0.588,
 'gpt-4o-agent': 0.72,
 'claude-3.5-sonnet-agent': 0.7166666666666667,
 'gemini-v1.5-flash-agent': 0.4,
 'geminipro-agent': 0.6415094339622641,
 'nvidia-llama31-70-agent': 0.488,
 'nvidia-llama31-405-agent': 0.436}

In [37]:
arena_hard_samples = {}
for model_name in ["gpt-4o", "nvidia-llama31"]:
    arena_hard_samples[model_name+"-agent"] = pd.read_json("eval_samples/arena_hard_50_{}-agent.jsonl".format(model_name), lines=True)

In [38]:
web_search_invoked_arena = {}
for model_name in arena_hard_samples.keys():
    web_search_invoked_arena[model_name] = float((arena_hard_samples[model_name]["search_done"]).mean())
web_search_invoked_arena

{'gpt-4o-agent': 0.08, 'nvidia-llama31-agent': 0.22}

In [42]:
arena_hard_results = {}
for model_name in ["gpt-4o", "llama31"]:
    arena_hard_results[model_name] = pd.read_json("eval_results/arena_hard_{}.jsonl".format(model_name), lines=True)

In [45]:
winrate = {}
for model_name in arena_hard_results.keys():
    model_winrate = float((arena_hard_results[model_name]["grade"] == "agent").mean())
    winrate[model_name] = model_winrate
winrate

{'gpt-4o': 0.46, 'llama31': 0.4}

In [46]:
winrate = {}
for model_name in arena_hard_results.keys():
    model_winrate = float((arena_hard_results[model_name]["grade"] == "base").mean())
    winrate[model_name] = model_winrate
winrate

{'gpt-4o': 0.54, 'llama31': 0.6}